In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import tqdm
import os
import re
import copy
import datetime
import numba
from numba import jit,prange
import regionmask
from shapely.geometry import Polygon, MultiPolygon, shape
import fiona

In [2]:
shpfname = "GIS/cb_2018_us_nation_20m.shp"

areafpref = "cropmap/crops/"
areafsuf = ".HarvestedAreaFraction.nc"
areavname = "Band1"

varfpref = "sheffield_vpd/Sacks_ZARC_fill_fill_120d/teste/"
varfsuf = ".all.nc"
varvname = "tempmean"

In [3]:
crop = "Soybeans"

In [4]:
areafname = areafpref + crop + areafsuf
varfname = varfpref + crop + varfsuf

In [5]:
areain = xr.open_dataset(areafname)[areavname]
areain

<xarray.DataArray 'Band1' (lat: 2160, lon: 4320)>
[9331200 values with dtype=float32]
Coordinates:
  * lat      (lat) float64 -89.96 -89.88 -89.79 -89.71 ... 89.79 89.87 89.96
  * lon      (lon) float64 -180.0 -179.9 -179.8 -179.7 ... 179.8 179.9 180.0
Attributes:
    long_name:     GDAL Band Number 1
    grid_mapping:  crs

In [6]:
varin = xr.open_dataset(varfname)[varvname]
varin = varin.rename({"latitude":"lat", "longitude":"lon"})
varin

<xarray.DataArray 'tempmean' (lat: 180, lon: 360)>
array([[-33.086641, -33.137909, -33.060869, ..., -33.240492, -33.194292,
        -33.059055],
       [-38.982688, -38.890599, -38.642072, ..., -38.986444, -38.944998,
        -38.947341],
       [-35.20735 , -35.115651, -34.953442, ..., -35.537426, -35.332495,
        -35.221457],
       ...,
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan]])
Coordinates:
  * lon      (lon) float32 -179.75 -178.74861 -177.74721 ... 178.7486 179.75
  * lat      (lat) float32 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5

In [7]:
shp = fiona.open(shpfname)
poly = MultiPolygon(shape(shp.next()["geometry"]))
shpmsk = regionmask.Regions(poly).mask(areain["lon"],areain["lat"])

# shpmsk.plot()

C:\Users\eriad\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FionaDeprecationWarning: Collection.__next__() is buggy and will be removed in Fiona 2.0. Switch to `next(iter(collection))`.
  


In [8]:
shpmsk = xr.where(shpmsk.notnull(),1,np.nan)
shpmsk

<xarray.DataArray 'region' (lat: 2160, lon: 4320)>
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])
Coordinates:
  * lat      (lat) float64 -89.96 -89.88 -89.79 -89.71 ... 89.79 89.87 89.96
  * lon      (lon) float64 -180.0 -179.9 -179.8 -179.7 ... 179.8 179.9 180.0

In [12]:
maskedarea = areain.where(shpmsk.notnull())
# maskedarea.plot()

In [10]:
vargrid = varin.interp(lat=maskedarea['lat'],lon=maskedarea['lon'],method = 'nearest')
vargrid

<xarray.DataArray 'tempmean' (lat: 2160, lon: 4320)>
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])
Coordinates:
  * lat      (lat) float64 -89.96 -89.88 -89.79 -89.71 ... 89.79 89.87 89.96
  * lon      (lon) float64 -180.0 -179.9 -179.8 -179.7 ... 179.8 179.9 180.0

In [11]:
np.sum(maskedarea*vargrid)/np.sum(maskedarea)

<xarray.DataArray ()>
array(22.00656828)